<a href="https://colab.research.google.com/github/nneibaue/ocean_explorer/blob/staging/explorer_official.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Ocean Data Explorer</h1>

In [2]:
# etsp stuff
import ocean
import plotting as op
import ocean_utils as utils

# Colab output stuff
# from google.colab import drive
# from google.colab import widgets
from IPython.display import display, HTML
import ipywidgets
from ipywidgets.embed import embed_minimal_html

# General
import os
import numpy as np
import random
import re
import pandas as pd
from importlib import reload

# Plotting
from cycler import cycler
import altair as alt
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
#from IPython import display, html
#Namespace class to keep things organized
class Namespace:
  def __init__(self, **kwargs):
    self.__dict__.update(**kwargs)

# Convenience function to reload github and reimport
def refresh_modules():
  reload(ocean)
  reload(op)
  reload(utils)

display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

# Analysis

In [3]:
#@title Data Import
refresh_modules()
#@markdown Enter drive path to folder containing Profiles:
EXPERIMENT_DIR = "deglitched_profiles" #@param{type:"string"}
#depth_path = "software_development/etsp/XRF data/25m" #@param {type:"string"}
#@markdown Enter elements separated by comma
ELEMENTS_OF_INTEREST = "Br,Ca,Cu,Fe,K,Cl,Mn,S,Si,Zn" #@param {type:"string"}
ELEMENTS_OF_INTEREST=ELEMENTS_OF_INTEREST.split(',')
ORBITALS = "K" #@param {type:"string"}
profiles = ocean.load_profiles(EXPERIMENT_DIR,
                               elements_of_interest=ELEMENTS_OF_INTEREST,
                               orbitals=[ORBITALS],
                               normalized=True)

ETSP_PS2: Successfully imported data for 750m
ETSP_PS2: Successfully imported data for 250m
ETSP_PS2: Successfully imported data for 400m
ETSP_PS2: Successfully imported data for 20m
ETSP_PS2: Successfully imported data for 80m
ETSP_PS2_profile-20200916T191637Z-001.zip is not a valid name for a Depth!
ETSP deglitch.xlsx is not a valid name for a Depth!
ETSP_PS2: Successfully imported data for 40m
ETSP_PS3: Successfully imported data for 165m
ETSP_PS3: Successfully imported data for 10m
ETSP_PS3: Successfully imported data for 50m
ETSP_PS3: Successfully imported data for 250m
ETSP_PS3: Successfully imported data for 900m
ETSP deglitch.xlsx is not a valid name for a Depth!
ETSP_PS3_profile-20200916T191640Z-001.zip is not a valid name for a Depth!
ETSP_PS3: Successfully imported data for 25m
ETSP_PS3: Successfully imported data for 40m
ETSP_PS3: Successfully imported data for 65m


## Data Preview



In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import altair as alt
import pandas as pd


class DetsumPlot:
    def __init__(self, detsum):
        self.graph_output = ipywidgets.Output()
        self.detsum = detsum
        self.avg = np.mean(self.detsum.data)
        self.min = np.nanmin(self.detsum.data)
        self.max = np.nanmax(self.detsum.data)
        self._default_layout = ipywidgets.Layout(width='230px')
        
        self.step = self.max/100
        
        self.isNoisy_checkbox = ipywidgets.Checkbox(description='Noisy', value=self.detsum.isNoisy)
        self.isNoisy_checkbox.observe(self.set_noise_flag, names='value')
        
        self.force_refresh_button = ipywidgets.Button(description='refresh')
        self.force_refresh_button.on_click(self.refresh_graph)
        
        
        self.update_max({'new': self.max})
        
    def set_noise_flag(self, val):
        utils.set_noisy_detsum_flag(self.detsum, val['new'])
        
        
    def refresh_graph(self, _):
        self.graph_output.clear_output(wait=True)
        with self.graph_output:
            im = ipywidgets.HTML(self.detsum.plot(base64=True, raw=True))
            display(im)
        
    def update_max(self, val):
        self.graph_output.clear_output(wait=True)
        vmax = float(val['new'])
        with self.graph_output:
            im = ipywidgets.HTML(self.detsum.plot(base64=True, vmax=vmax, raw=True))
            display(im)
                                 
    def update_min(self, val):
        self.graph_output.clear_output(wait=True)
        vmin = float(val['new'])
        with self.graph_output:
            im = ipywidgets.HTML(self.detsum.plot(base64=True, vmin=vmin, raw=True))
            display(im)
            
        
    @property
    def widget(self):
        self.max_slider = ipywidgets.FloatSlider(layout=self._default_layout,
                                             min=self.step, max=2*self.max, value=self.max, step=self.step,
                                             readout_format='.2e', continuous_update=False)
        
        self.min_slider = ipywidgets.FloatSlider(layout=self._default_layout,
                                             min=self.step, max=2*self.max, value=self.step, step=self.step,
                                             readout_format='.2e', continuous_update=False)
        
        self.max_slider.observe(self.update_max, names='value')
        self.min_slider.observe(self.update_min, names='value')
        min_label = ipywidgets.Text(f'min counts: {self.min:0.2e}')
        max_label = ipywidgets.Text(f'max counts: {self.max:0.2e}')
        avg_label = ipywidgets.Text(f'avg counts: {self.avg:0.2e}')
        labels = ipywidgets.HTML(f'''<center><p>min: {self.min:0.2e}</p>
                                     <p>max: {self.max:0.2e}</p>
                                     <p>avg: {self.avg:0.2e}</p></center>''',
                                layout=self._default_layout)
        #labels = ipywidgets.VBox([min_label, max_label, avg_label], layout=self._default_layout)
        
        vbox = ipywidgets.VBox([self.graph_output,
                                self.min_slider,
                                self.max_slider,
                                labels,
                                ipywidgets.HBox([self.force_refresh_button, self.isNoisy_checkbox])])
        return vbox
        
        
class DepthPlot:
    def __init__(self, depth):
        self._tab_dict = {}
        for scan in depth.scans:
            plots = [DetsumPlot(d).widget for d in scan.detsums]
            self._tab_dict[scan.name] = ipywidgets.HBox(plots)
            
    @property
    def widget(self):
        children = []
        tabs = ipywidgets.Tab()
        for i, scan_name in enumerate(self._tab_dict):
            tabs.set_title(i, scan_name)
            children.append(self._tab_dict[scan_name])
            
        tabs.children = children
        return tabs

    
class NoiseFlaggingUI:
    def __init__(self, profile):
        self._tab_dict = {}
        for depth in profile.depths:
            plots = [DetsumPlot(d).widget for d in depth.detsums]
            rows = []
            for i in np.arange(0, len(plots), 4):
                rows.append(ipywidgets.HBox(plots[i:i+4]))
                
            self._tab_dict[depth.depth] = ipywidgets.VBox(rows)
            
            
    @property
    def widget(self):
        children = []
        tabs = ipywidgets.Tab()
        for i, depth in enumerate(self._tab_dict):
            tabs.set_title(i, depth)
            children.append(self._tab_dict[depth])
            
        tabs.children = children
        return tabs

    
    


nf = NoiseFlaggingUI(profiles['ETSP_PS2'])
#embed_minimal_html('noiseFlaggingUI.html', views=[nf.widget], title='Noise Flagging')
display(nf.widget)


/Users/naomi/xraylarch/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: All-NaN slice encountered
  if sys.path[0] == '':
/Users/naomi/xraylarch/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: All-NaN slice encountered
  del sys.path[0]


## **Plotting UIs**

In [ ]:
list(profiles.keys())

In [ ]:
#@title Ribbon Plot UI { form-width: "200px" }
run_ribbon_plot_ui = True #@param {type:"boolean"}
PROFILE_NAME = "ETSP_PS2" #@param {type:"string"}

from ipywidgets.embed import embed_minimal_html
def ribbon_plot_ui(profile_name):
  profile = profiles[profile_name]
  experiment_dir = profile.experiment_dir
  status_indicator = ipywidgets.Output()
  with status_indicator:
    display(ipywidgets.HTML('<h3 style="color:green">Ready</h3>'))
  graph_output = ipywidgets.Output()
  # element_inputs = {}
  # element_filter = {}
  test = {}
  smalltextbox = ipywidgets.Layout(width='50px', height='25px')
  # filter_func = lambda n: lambda x: np.mean(x) + np.std(x)*n
  
  element_filter = op.ElementFilterPanel(profile,
                                    input_type='text',
                                    orientation='horizontal',
                                    experiment_dir=experiment_dir)#, **layout_kwargs)
  filter_settings = op.SettingsController(element_filter)
  
  # for e in ELEMENTS_OF_INTEREST:
  #   element_inputs[e] = ipywidgets.Textarea(value='2', layout=smalltextbox)
  #   element_filter[e] = filter_func(2)
    
  filter_by_control = ipywidgets.Dropdown(options=ELEMENTS_OF_INTEREST,
                                          value='Cu', description='Filter by:',
                                          layout=ipywidgets.Layout(width='200px'))
  
  combine_scans_checkbox = ipywidgets.Checkbox(value=True, description='Combine Scans')
  
  combine_detsums_checkbox = ipywidgets.Checkbox(value=False, description='Combine Detsums')
  
  normalize_by_control = ipywidgets.Dropdown(options=['counts', 'pixels'],
                                            value='counts',
                                            description='Normalize By',
                                            layout=ipywidgets.Layout(width='200px'))
  
  
  N_input = ipywidgets.Textarea(value='8', layout=ipywidgets.Layout(width='150px'), description='N')
  update_button = ipywidgets.Button(description='Update Plot')                          
  clear_output_control = ipywidgets.Checkbox(value=False, description='Clear output after each run')
  
  # element_filter_input = ipywidgets.HBox(
  #     [ipywidgets.VBox([ipywidgets.HTML(f'<h3>{e}</h3>'), element_inputs[e]]) for e in ELEMENTS_OF_INTEREST]
  # )
  
  save_html_button = ipywidgets.Button(description='Save HTML')
  def update_plot(b):
    element_filter.save_settings()
    status_indicator.clear_output()
    # for e in ELEMENTS_OF_INTEREST:
    #   val = float(element_inputs[e].value)
    #   element_filter[e] = filter_func(val)
    with status_indicator:
      display(ipywidgets.HTML('<h3 style="color:red">Working...</h3>'))

    info_banner_html = (f'Filter by: {filter_by_control.value} | '
                      f'Comb. Scans: {combine_scans_checkbox.value} | '
                      f'Comb. Detsums: {combine_detsums_checkbox.value} | '
                      f'N: {N_input.value} | '
                      f'Normalize By: {normalize_by_control.value} | ')
    info_banner = ipywidgets.HTML(info_banner_html)

    plot = op.ribbon_plot(profile, element_filter=element_filter.filter_dict,
                filter_by=filter_by_control.value,
                combine_detsums=combine_detsums_checkbox.value,
                combine_scans=combine_scans_checkbox.value,
                N=int(N_input.value),
                normalize_by=normalize_by_control.value,
                base64=True)
    if clear_output_control.value:
      graph_output.clear_output()

    with graph_output:
      #display(ipywidgets.HTML(plot))
      display(ipywidgets.VBox([ipywidgets.HTML(plot), ipywidgets.HTML(info_banner_html)]))

    status_indicator.clear_output()
    with status_indicator:
      display(ipywidgets.HTML('<h3 style="color:green">Ready</h3>'))
  
  update_button.on_click(update_plot)
  

  update_plot('this param does not matter here')  

  #https://stackoverflow.com/questions/55336771/align-ipywidget-button-to-center
  controls_bot = ipywidgets.Box([element_filter.widget, filter_settings.widget],
                                layout=ipywidgets.Layout(display='flex', align_items='center'))
  controls_top = ipywidgets.HBox([ipywidgets.VBox([update_button, status_indicator]),
                              ipywidgets.VBox([filter_by_control, normalize_by_control]),
                              ipywidgets.VBox([combine_scans_checkbox, combine_detsums_checkbox]),
                              N_input, clear_output_control])

  controls = ipywidgets.VBox([controls_top, controls_bot],
                            layout=ipywidgets.Layout(
                                border='1px solid black',
                                width='100%',
                            ))
  app = ipywidgets.VBox([graph_output, controls])
  display(app)
  return filter_settings

  # def save_html(b):
  #   embed_minimal_html(os.path.join(experiment_dir, 'myplots.html'), views=[graph_output], title='Ribbon Plot Examples')

  # save_html_button.on_click(save_html)
if run_ribbon_plot_ui:    
  a = ribbon_plot_ui(PROFILE_NAME)

In [ ]:
#@title Image UI { form-width: "200px" }
run_image_ui = True #@param {type:"boolean"}

PROFILE_NAME = 'ETSP_PS3' #@param {type:"string"}
def image_ui(profile_name):
  profile = profiles[profile_name]
  experiment_dir = profile.experiment_dir
  plot_area = ipywidgets.Output()
  status_indicator = ipywidgets.Output()
  group_indicator = ipywidgets.Output()
  
  with group_indicator:
    display(ipywidgets.HTML('<h3 style="color:orange">No Group Selected</h3>'))

  with status_indicator:
    display(ipywidgets.HTML('<h3 style="color:green">Ready</h3>'))

  
  # Making the controls

  layout_kwargs = dict(width='85%', border='1px solid black')
  #settings_layout = dict(width='20%', border='1px solid blue')

  depth_selector = op.PropSelector(profile.depths, orientation='horizontal', title='Depths to plot',
                                   description_func=lambda d: d.depth, **layout_kwargs)
  element_filter = op.ElementFilterPanel(profile,
                                    orientation='horizontal',
                                    experiment_dir=experiment_dir)

  filter_settings = op.SettingsController(element_filter)

  element_plot_selector = op.PropSelector(ELEMENTS_OF_INTEREST,
                                          orientation='horizontal',
                                          title='Elements to plot',
                                          **layout_kwargs)

  element_group_selector = op.PropSelector(ELEMENTS_OF_INTEREST,
                                           orientation='horizontal',
                                           title='Groups to show',
                                           **layout_kwargs)

  combine_detsums_checkbox = ipywidgets.Checkbox(value=False, indent=False, description='Combine Detsums')
  update_button = ipywidgets.Button(description='Update')
  raw_data_toggle = ipywidgets.ToggleButtons(value='Filtered', options=['Filtered', 'Raw'])
  show_groups_toggle = ipywidgets.ToggleButton(value=True, description='Show Groups')
  exclusive_groups_toggle = ipywidgets.ToggleButtons(value='Exclusive', options=['Exclusive', 'Nonexclusive'])

  controls_bottom = ipywidgets.HBox([update_button,
                                          show_groups_toggle,
                                          exclusive_groups_toggle,
                                          raw_data_toggle,
                                          #combine_detsums_checkbox,
                                          status_indicator],
                                         layout=ipywidgets.Layout(padding='5px', **layout_kwargs))


  controls_right = ipywidgets.HBox([
      element_filter.widget, ipywidgets.VBox([
        combine_detsums_checkbox,
        filter_settings.widget])
      ],
      layout=ipywidgets.Layout(**layout_kwargs))
  controls_top = ipywidgets.VBox([depth_selector.widget,
                                   element_plot_selector.widget,
                                   controls_right,
                                   #ipywidgets.HBox([group_indicator, show_groups_toggle]),
                                   element_group_selector.widget,
                                   group_indicator])

  # controls_right = ipywidgets.VBox([ipywidgets.HTML('Hightlight group\ncontaining elements:'),
  #                                   element_group_selector.widget],
  #                                  layout=ipywidgets.Layout(border='1px solid black'))

  # controls = ipywidgets.HBox([controls_left, controls_right], layout=ipywidgets.Layout(width='85%'))
  controls = ipywidgets.VBox([controls_top, controls_bottom])
                            
  rows = []
  rows_raw = []
  rows_groups_exclusive = []
  rows_groups_exclusive_raw = []
  rows_groups_nonexclusive = []
  rows_groups_nonexclusive_raw = []
  
  current_group = []


  def show_plots(val):
    plot_area.clear_output()
    show = show_groups_toggle.value
    exclusive = exclusive_groups_toggle.value == 'Exclusive'
    with plot_area:
      if raw_data_toggle.value == 'Raw':
        if show and exclusive:
          display(ipywidgets.VBox(rows_groups_exclusive_raw))
        elif show and not exclusive:
          display(ipywidgets.VBox(rows_groups_nonexclusive_raw))
        elif not show:
          display(ipywidgets.VBox(rows_raw))
      elif raw_data_toggle.value == 'Filtered':
        if show and exclusive:
          display(ipywidgets.VBox(rows_groups_exclusive))
        if show and not exclusive:
          display(ipywidgets.VBox(rows_groups_nonexclusive))
        elif not show:
          display(ipywidgets.VBox(rows))
        
    status_indicator.clear_output()
    with status_indicator:
      display(ipywidgets.HTML('<h3 style="color:green">Ready</h3>'))


  def update_group(element, val):
    nonlocal current_group
    if val:
      current_group.append(element)
    elif not val:
      current_group.remove(element)

    sorted_group = [ELEMENTS_OF_INTEREST[ELEMENTS_OF_INTEREST.index(element)] if
                                          element in current_group else None for element in ELEMENTS_OF_INTEREST]
    sorted_group = list(filter(lambda x: x, sorted_group))
    group_indicator.clear_output()
    with group_indicator:
      if not current_group:
        display(ipywidgets.HTML('<h3 style="color:orange">No group selected</h3>'))
      else:
        group_str = '  |  '.join(sorted_group)
        display(ipywidgets.HTML(f'<h3 style="color:red">Group Selected: {group_str}</hp>'))
    current_group = sorted_group

  def generate_plots(b):
    element_filter.save_settings()
    status_indicator.clear_output()
    with status_indicator:
      display(ipywidgets.HTML('<h3 style="color:red">Working....</h3>'))

    depths_to_plot = depth_selector.selected_props
    if not depths_to_plot:
      status_indicator.clear_output()
      with status_indicator:
        display(ipywidgets.HTML('<h3 style="color:orange">No Depth Selected!</h3>'))
      return

    elements_to_plot = element_plot_selector.selected_props

    group = '|'.join(element_group_selector.selected_props)

    for depth in depths_to_plot:
      depth.apply_element_filter(element_filter.filter_dict[depth.depth],
                                 combine_detsums=combine_detsums_checkbox.value)

    def get_row(depth, elements, raw, show_groups, exclusive):
      detsums = sorted(depth.detsums, key=lambda d: d.element)
      plots = []
      #group = '|'.join(current_group)
      for scan in depth.scans:
        data = scan.data['element_group'].values.reshape(scan.detsums[0].shape)
        for detsum in scan.detsums:
          if detsum.element not in elements:
            continue
          fig, ax = plt.subplots(figsize=(15, 15))
          detsum.plot(raw=raw, ax=ax)
          if show_groups and current_group:
            fn = np.vectorize(lambda group: utils.check_groups(group, current_group, exclusive=exclusive))
            rows, cols = np.where(fn(data))
            ax.scatter(cols, rows, s=20, color='red')
          plot = op.encode_matplotlib_fig(fig)
          plt.close()
          plots.append(ipywidgets.HTML(plot))
      return plots

    nonlocal rows
    nonlocal rows_raw
    nonlocal rows_groups_exclusive
    nonlocal rows_groups_exclusive_raw
    nonlocal rows_groups_nonexclusive
    nonlocal rows_groups_nonexclusive_raw

    
    rows = [ipywidgets.HBox(get_row(depth, elements_to_plot, False, False, False)) for depth in depths_to_plot]
    rows_raw = [ipywidgets.HBox(get_row(depth, elements_to_plot, True, False, False)) for depth in depths_to_plot]
    rows_groups_exclusive = [ipywidgets.HBox(get_row(depth, elements_to_plot, False, True, True)) for depth in depths_to_plot]
    rows_groups_exclusive_raw = [ipywidgets.HBox(get_row(depth, elements_to_plot, True, True, True)) for depth in depths_to_plot]
    rows_groups_nonexclusive = [ipywidgets.HBox(get_row(depth, elements_to_plot, False, True, False)) for depth in depths_to_plot]
    rows_groups_nonexclusive_raw = [ipywidgets.HBox(get_row(depth, elements_to_plot, True, True, False)) for depth in depths_to_plot]

    show_plots(_)


  update_button.on_click(generate_plots)
  raw_data_toggle.observe(show_plots, 'value')
  show_groups_toggle.observe(show_plots, 'value')
  exclusive_groups_toggle.observe(show_plots, 'value')
  element_group_selector.observe(update_group)
  # for selector in depth_selectors.values():
  #   selector.observe(update_plot)
  display(ipywidgets.VBox([controls, plot_area]))
  #print('\n'.join(dir(plot_area)))
  
  
if run_image_ui:
  image_ui(PROFILE_NAME)